In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pandas as pd

from sklearn import preprocessing
from sklearn.cluster import KMeans, DBSCAN #importamos kmeans y dbsac de la libreria de clustering de sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler #para escalar
from sklearn.preprocessing import MinMaxScaler

In [2]:
# parametros (cosas que son variables pero que para todo mi jupyter cuando corra sera constante)

signalcrop=(0,300)
signaltype="crop" #se puede cambiar por "raw", "procesada"
asign_labels={'12_12 led':None, 'DD4':0, 'DP':None, 'LDR':0, 'LL2':1, 'LL3':1, 'LL4':2, 'LL6':2, 'LL8':2} #esto es el diccionario


In [3]:
#dataframe principal
main_df = pd.read_excel('Dataframe_integral.xlsx',index_col=0)

In [4]:
#señal cruda
signal_df= main_df.filter(items=["Exposicion"] + [i for i in range(1,511)])
signal_df

,Exposicion,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,506,507,508,509,510
0,DD4,-5174,-6696,-8410,-10073,-11377,-12239,-12896,-13680,-14442,...,-29282,-29104,-29103,-29464,-30053,-30788,-31599,-32285,-32746,-33017
1,DD4,-7394,-8885,-10538,-11996,-13043,-13718,-14306,-14874,-15329,...,-31612,-31454,-31532,-31941,-32568,-33381,-34228,-34787,-34916,-34688
2,DD4,-19334,-19300,-19115,-18777,-18461,-18265,-18082,-17813,-17542,...,-35477,-35457,-35330,-35157,-35060,-35086,-35073,-34964,-34758,-34479
3,DD4,-2017,-1777,-1505,-1204,-919,-741,-711,-914,-1416,...,-35511,-35504,-35371,-35112,-34762,-34323,-33818,-33365,-33028,-32792
4,DD4,-7555,-7931,-8090,-7882,-7378,-6774,-6175,-5789,-5970,...,-86266,-85864,-85356,-84842,-84403,-84184,-84129,-84142,-84169,-84187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,LL8,1714,2017,1707,864,-47,-657,-848,-763,-388,...,-1394,-2084,-2292,-2117,-1713,-1258,-778,-231,403,1105
200,LL8,-9150,-8352,-7330,-6210,-5185,-4440,-4044,-3978,-4257,...,829,134,346,1793,4117,5879,5654,3425,-386,-5279
201,LL8,-7317,-6922,-6307,-5519,-4724,-4149,-3866,-3950,-4340,...,-1712,-2142,-1152,1080,3431,4268,2988,-5,-4178,-9101
202,LL8,8773,8829,8864,8757,8482,8133,7779,7478,7277,...,-8518,-10888,-12758,-12833,-11486,-10354,-10603,-11878,-13687,-15775


In [5]:
#dataframe con estadisticos
main_mat = signal_df.filter([i for i in range(1, 511)]).values  #seria el pdData como tiene Toto
stats_mat = np.concatenate([main_mat.mean(axis=1)[:, np.newaxis],
                           main_mat.min(axis=1)[:, np.newaxis],
                           main_mat.max(axis=1)[:, np.newaxis],
                           main_mat.std(axis=1)[:, np.newaxis],
                           (main_mat**2).mean(axis=1)[:, np.newaxis]], axis=1) #parametro energia

stats_df=main_df.filter(items=["Exposicion"]).join(pd.DataFrame(stats_mat, columns=["mean","min","max","std","energy"]))
stats_df

,Exposicion,mean,min,max,std,energy
0,DD4,-1231.715686,-33017.0,49859.0,24058.203333,5.803143e+08
1,DD4,-1377.088235,-34916.0,52395.0,25219.938692,6.379417e+08
2,DD4,-1347.425490,-48690.0,83148.0,42119.165518,1.775840e+09
3,DD4,-909.574510,-46218.0,61078.0,35317.945480,1.248185e+09
4,DD4,-1991.337255,-100387.0,125801.0,77320.639554,5.982447e+09
...,...,...,...,...,...,...
199,LL8,-13.911765,-16896.0,23805.0,10581.002471,1.119578e+08
200,LL8,-597.815686,-34748.0,15157.0,12947.789862,1.680026e+08
201,LL8,-562.643137,-31364.0,14112.0,11521.960620,1.330721e+08
202,LL8,1033.386275,-23637.0,31802.0,15125.795055,2.298576e+08


In [6]:
stats_df.to_excel('stats_df.xlsx')

In [7]:
#dataframe con amplitud, latencia, ancho, pendiente

wave_df= main_df.filter(items=["Exposicion"] + [c for c in main_df.columns if "ONDA" in str(c) or "PENDIENTE" in str(c)])
wave_df

,Exposicion,LATENCIA ONDA B,AMPLITUD ONDA B,ANCHO ONDA B,LATENCIA ONDA A,AMPLITUD ONDA A,ANCHO ONDA A,LATENCIA ONDA B FILT.,AMPLITUD ONDA B FILT.,ANCHO ONDA B FILT.,LATENCIA ONDA A FILT.,AMPLITUD ONDA A FILT.,ANCHO ONDA A FILT.,PENDIENTE AB
0,DD4,178,55033,176.062539,69,25565,88.000000,182,32266.528145,160.240512,65,27480.172908,103.279638,510.655565
1,DD4,178,59789,173.437930,69,25708,87.053982,182,33539.967193,158.427521,64,28738.807299,104.384110,527.786225
2,DD4,165,102482,178.001588,56,167,19.844146,157,51774.199421,160.307473,60,21106.803675,86.337587,751.350547
3,DD4,146,63095,183.586019,68,22439,78.878717,131,33494.928154,171.712986,67,32374.607143,96.925661,1029.211489
4,DD4,118,133356,217.014391,42,41274,41.543385,127,83079.998260,180.074044,34,42545.774439,58.984510,1350.814760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,LL8,153,22091,172.455981,95,15729,96.972384,219,15277.856590,150.476625,100,14801.081805,133.001323,252.764188
200,LL8,89,20624,202.373244,19,5182,28.869681,206,27352.912427,211.090677,15,2035.357620,27.774400,153.865288
201,LL8,88,17775,205.844172,18,7503,29.387892,207,26965.804871,214.843079,16,3276.702142,30.079195,158.337733
202,LL8,91,23029,104.292029,11,1646,21.376623,107,27299.078492,135.325759,15,1489.483603,27.764172,312.919153


In [8]:
wave_df.to_excel('wave_df.xlsx')

## Escalado
Los scaler funcionan bien para stats y wave. Para signal aplicamos otro método.
Para wave_df conviene aplicar minmax en lugar de standard, porque me llevaria la pendiente 0 a negativo (escala entre 1 y -1)
Para stats_df conviene aplicar standard
Referencia: https://scikit-learn.org/stable/modules/preprocessing.html

In [9]:
#Quito la fila exposicion a stats_df
stats_df_for_transformed = stats_df.copy().dropna() #el dropna lo que hace es quitar donde tenia None en el dict
stats_df_for_transformed.pop("Exposicion") #le quita la columna llamada Exposicion

stats_df_for_transformed.index=np.arange(stats_df_for_transformed.values.shape[0]) #devuelve matriz de 175 valores

stats_df_for_transformed

,mean,min,max,std,energy
0,-1231.715686,-33017.0,49859.0,24058.203333,5.803143e+08
1,-1377.088235,-34916.0,52395.0,25219.938692,6.379417e+08
2,-1347.425490,-48690.0,83148.0,42119.165518,1.775840e+09
3,-909.574510,-46218.0,61078.0,35317.945480,1.248185e+09
4,-1991.337255,-100387.0,125801.0,77320.639554,5.982447e+09
...,...,...,...,...,...
199,-13.911765,-16896.0,23805.0,10581.002471,1.119578e+08
200,-597.815686,-34748.0,15157.0,12947.789862,1.680026e+08
201,-562.643137,-31364.0,14112.0,11521.960620,1.330721e+08
202,1033.386275,-23637.0,31802.0,15125.795055,2.298576e+08


In [10]:
#Quito la fila exposicion a wave_df
wave_df_for_transformed = wave_df.copy().dropna() #el dropna lo que hace es quitar donde tenia None en el dict
wave_df_for_transformed.pop("Exposicion") #le quita la columna llamada Exposicion

wave_df_for_transformed.index=np.arange(wave_df_for_transformed.values.shape[0]) #devuelve matriz de 175 valores

wave_df_for_transformed

,LATENCIA ONDA B,AMPLITUD ONDA B,ANCHO ONDA B,LATENCIA ONDA A,AMPLITUD ONDA A,ANCHO ONDA A,LATENCIA ONDA B FILT.,AMPLITUD ONDA B FILT.,ANCHO ONDA B FILT.,LATENCIA ONDA A FILT.,AMPLITUD ONDA A FILT.,ANCHO ONDA A FILT.,PENDIENTE AB
0,178,55033,176.062539,69,25565,88.000000,182,32266.528145,160.240512,65,27480.172908,103.279638,510.655565
1,178,59789,173.437930,69,25708,87.053982,182,33539.967193,158.427521,64,28738.807299,104.384110,527.786225
2,165,102482,178.001588,56,167,19.844146,157,51774.199421,160.307473,60,21106.803675,86.337587,751.350547
3,146,63095,183.586019,68,22439,78.878717,131,33494.928154,171.712986,67,32374.607143,96.925661,1029.211489
4,118,133356,217.014391,42,41274,41.543385,127,83079.998260,180.074044,34,42545.774439,58.984510,1350.814760
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,153,22091,172.455981,95,15729,96.972384,219,15277.856590,150.476625,100,14801.081805,133.001323,252.764188
200,89,20624,202.373244,19,5182,28.869681,206,27352.912427,211.090677,15,2035.357620,27.774400,153.865288
201,88,17775,205.844172,18,7503,29.387892,207,26965.804871,214.843079,16,3276.702142,30.079195,158.337733
202,91,23029,104.292029,11,1646,21.376623,107,27299.078492,135.325759,15,1489.483603,27.764172,312.919153


In [11]:
#Quito la fila exposicion a signal_df
signal_df_for_transformed = signal_df.copy().dropna() #el dropna lo que hace es quitar donde tenia None en el dict
signal_df_for_transformed.pop("Exposicion") #le quita la columna llamada Exposicion

signal_df_for_transformed.index=np.arange(signal_df_for_transformed.values.shape[0]) #devuelve matriz de 175 valores

signal_df_for_transformed

,1,2,3,4,5,6,7,8,9,10,...,501,502,503,504,505,506,507,508,509,510
0,-5174,-6696,-8410,-10073,-11377,-12239,-12896,-13680,-14442,-15036,...,-29282,-29104,-29103,-29464,-30053,-30788,-31599,-32285,-32746,-33017
1,-7394,-8885,-10538,-11996,-13043,-13718,-14306,-14874,-15329,-15756,...,-31612,-31454,-31532,-31941,-32568,-33381,-34228,-34787,-34916,-34688
2,-19334,-19300,-19115,-18777,-18461,-18265,-18082,-17813,-17542,-17341,...,-35477,-35457,-35330,-35157,-35060,-35086,-35073,-34964,-34758,-34479
3,-2017,-1777,-1505,-1204,-919,-741,-711,-914,-1416,-2273,...,-35511,-35504,-35371,-35112,-34762,-34323,-33818,-33365,-33028,-32792
4,-7555,-7931,-8090,-7882,-7378,-6774,-6175,-5789,-5970,-6722,...,-86266,-85864,-85356,-84842,-84403,-84184,-84129,-84142,-84169,-84187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1714,2017,1707,864,-47,-657,-848,-763,-388,181,...,-1394,-2084,-2292,-2117,-1713,-1258,-778,-231,403,1105
200,-9150,-8352,-7330,-6210,-5185,-4440,-4044,-3978,-4257,-4691,...,829,134,346,1793,4117,5879,5654,3425,-386,-5279
201,-7317,-6922,-6307,-5519,-4724,-4149,-3866,-3950,-4340,-4857,...,-1712,-2142,-1152,1080,3431,4268,2988,-5,-4178,-9101
202,8773,8829,8864,8757,8482,8133,7779,7478,7277,7199,...,-8518,-10888,-12758,-12833,-11486,-10354,-10603,-11878,-13687,-15775


In [12]:
#standard scaler para dataframe estadisticos
ss = StandardScaler()
stats_df_transformed_ss = ss.fit_transform(stats_df_for_transformed) #aplicado a dataframe estadisticos
stats_df_ready=pd.DataFrame(stats_df_transformed_ss, columns=["mean","min","max","std","energy"])
stats_df_ready

,mean,min,max,std,energy
0,-0.573769,0.333366,0.015322,-0.089710,-0.289680
1,-0.672474,0.242211,0.098394,-0.018877,-0.240977
2,-0.652334,-0.418959,1.105769,1.011489,0.720702
3,-0.355042,-0.300300,0.382823,0.596810,0.274762
4,-1.089536,-2.900485,2.502953,3.157765,4.275857
...,...,...,...,...,...
199,0.253094,1.107195,-0.838128,-0.911431,-0.685505
200,-0.143364,0.250275,-1.121410,-0.767125,-0.638139
201,-0.119483,0.412712,-1.155641,-0.854059,-0.667660
202,0.964187,0.783618,-0.576171,-0.634329,-0.585863


In [13]:
stats_df_ready.to_excel('stats_df_ready.xlsx')

In [14]:
stats_df_for_etiqueta=stats_df.filter(items=['Exposicion']).join(stats_df_ready)
stats_df_for_etiqueta

,Exposicion,mean,min,max,std,energy
0,DD4,-0.573769,0.333366,0.015322,-0.089710,-0.289680
1,DD4,-0.672474,0.242211,0.098394,-0.018877,-0.240977
2,DD4,-0.652334,-0.418959,1.105769,1.011489,0.720702
3,DD4,-0.355042,-0.300300,0.382823,0.596810,0.274762
4,DD4,-1.089536,-2.900485,2.502953,3.157765,4.275857
...,...,...,...,...,...,...
199,LL8,0.253094,1.107195,-0.838128,-0.911431,-0.685505
200,LL8,-0.143364,0.250275,-1.121410,-0.767125,-0.638139
201,LL8,-0.119483,0.412712,-1.155641,-0.854059,-0.667660
202,LL8,0.964187,0.783618,-0.576171,-0.634329,-0.585863


In [15]:
#min max scaler para dataframe con amplitud, latencia, ancho, pendiente
scaler = MinMaxScaler()
wave_df_transformed_MinMax = scaler.fit_transform(wave_df_for_transformed)
wave_df_ready=pd.DataFrame(wave_df_transformed_MinMax, columns=[c for c in main_df.columns if "ONDA" in str(c) or "PENDIENTE" in str(c)])
wave_df_ready

,LATENCIA ONDA B,AMPLITUD ONDA B,ANCHO ONDA B,LATENCIA ONDA A,AMPLITUD ONDA A,ANCHO ONDA A,LATENCIA ONDA B FILT.,AMPLITUD ONDA B FILT.,ANCHO ONDA B FILT.,LATENCIA ONDA A FILT.,AMPLITUD ONDA A FILT.,ANCHO ONDA A FILT.,PENDIENTE AB
0,0.622378,0.306643,0.712617,0.403509,0.313619,0.393907,0.645390,0.245408,0.678310,0.305164,0.341063,0.426210,0.190702
1,0.622378,0.333144,0.701994,0.403509,0.315374,0.389673,0.645390,0.255094,0.670636,0.300469,0.356684,0.430768,0.197522
2,0.576923,0.571029,0.720466,0.327485,0.002049,0.088827,0.556738,0.393777,0.678594,0.281690,0.261962,0.356294,0.286526
3,0.510490,0.351565,0.743069,0.397661,0.275271,0.353079,0.464539,0.254751,0.726874,0.314554,0.401809,0.399988,0.397147
4,0.412587,0.743059,0.878371,0.245614,0.506330,0.185957,0.450355,0.631879,0.762267,0.159624,0.528046,0.243415,0.525183
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0.534965,0.123091,0.698020,0.555556,0.192956,0.434070,0.776596,0.116198,0.636979,0.469484,0.183700,0.548864,0.088031
200,0.311189,0.114917,0.819111,0.111111,0.063570,0.129227,0.730496,0.208037,0.893563,0.070423,0.025261,0.114618,0.048657
201,0.307692,0.099042,0.833159,0.105263,0.092043,0.131547,0.734043,0.205093,0.909447,0.075117,0.040668,0.124129,0.050438
202,0.318182,0.128317,0.422125,0.064327,0.020192,0.095686,0.379433,0.207628,0.572844,0.070423,0.018486,0.114576,0.111979


In [16]:
wave_df_ready.to_excel('wave_df_ready.xlsx')

In [17]:
wave_df_for_etiqueta=wave_df.filter(items=['Exposicion']).join(wave_df_ready)
wave_df_for_etiqueta

,Exposicion,LATENCIA ONDA B,AMPLITUD ONDA B,ANCHO ONDA B,LATENCIA ONDA A,AMPLITUD ONDA A,ANCHO ONDA A,LATENCIA ONDA B FILT.,AMPLITUD ONDA B FILT.,ANCHO ONDA B FILT.,LATENCIA ONDA A FILT.,AMPLITUD ONDA A FILT.,ANCHO ONDA A FILT.,PENDIENTE AB
0,DD4,0.622378,0.306643,0.712617,0.403509,0.313619,0.393907,0.645390,0.245408,0.678310,0.305164,0.341063,0.426210,0.190702
1,DD4,0.622378,0.333144,0.701994,0.403509,0.315374,0.389673,0.645390,0.255094,0.670636,0.300469,0.356684,0.430768,0.197522
2,DD4,0.576923,0.571029,0.720466,0.327485,0.002049,0.088827,0.556738,0.393777,0.678594,0.281690,0.261962,0.356294,0.286526
3,DD4,0.510490,0.351565,0.743069,0.397661,0.275271,0.353079,0.464539,0.254751,0.726874,0.314554,0.401809,0.399988,0.397147
4,DD4,0.412587,0.743059,0.878371,0.245614,0.506330,0.185957,0.450355,0.631879,0.762267,0.159624,0.528046,0.243415,0.525183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,LL8,0.534965,0.123091,0.698020,0.555556,0.192956,0.434070,0.776596,0.116198,0.636979,0.469484,0.183700,0.548864,0.088031
200,LL8,0.311189,0.114917,0.819111,0.111111,0.063570,0.129227,0.730496,0.208037,0.893563,0.070423,0.025261,0.114618,0.048657
201,LL8,0.307692,0.099042,0.833159,0.105263,0.092043,0.131547,0.734043,0.205093,0.909447,0.075117,0.040668,0.124129,0.050438
202,LL8,0.318182,0.128317,0.422125,0.064327,0.020192,0.095686,0.379433,0.207628,0.572844,0.070423,0.018486,0.114576,0.111979


In [18]:
#para escalar el signal_df

def samples_escaler(pandas):
    nuevo_pandas=pandas.copy()
    for i in range(pandas.values.shape[0]): #otro forma es for i in pandas.index
        media=pandas.iloc[i].mean()
        desvio=pandas.iloc[i].std()
        nuevo_pandas.iloc[i]=pandas.iloc[i].map(lambda x:(x-media)/desvio) #esto es para estandarizar normalizando

        #print(media,desvio)
        #print(nuevo_pandas.iloc[i].mean(),nuevo_pandas.iloc[i].std())
    
    return(nuevo_pandas)

In [19]:
signal_df_scaler=samples_escaler(signal_df_for_transformed)

In [20]:
signal_df_scaler #dataframe signal escalado

#este df lo voy a exportar asi, no tiene sentido elegir algunos features aca.

,1,2,3,4,5,6,7,8,9,10,...,501,502,503,504,505,506,507,508,509,510
0,-0.163704,-0.226905,-0.298079,-0.367135,-0.421284,-0.457079,-0.484360,-0.516916,-0.548558,-0.573224,...,-1.164791,-1.157399,-1.157358,-1.172348,-1.196807,-1.227327,-1.261004,-1.289491,-1.308634,-1.319887
1,-0.238344,-0.297405,-0.362885,-0.420639,-0.462113,-0.488852,-0.512144,-0.534643,-0.552667,-0.569581,...,-1.197674,-1.191415,-1.194505,-1.210706,-1.235543,-1.267748,-1.301299,-1.323443,-1.328553,-1.319521
2,-0.426621,-0.425815,-0.421427,-0.413410,-0.405915,-0.401266,-0.396925,-0.390545,-0.384117,-0.379350,...,-0.809515,-0.809041,-0.806028,-0.801925,-0.799624,-0.800241,-0.799933,-0.797347,-0.792461,-0.785844
3,-0.031325,-0.024536,-0.016842,-0.008328,-0.000267,0.004768,0.005617,-0.000125,-0.014325,-0.038566,...,-0.978751,-0.978553,-0.974791,-0.967465,-0.957565,-0.945147,-0.930862,-0.918049,-0.908516,-0.901840
4,-0.071885,-0.076743,-0.078798,-0.076110,-0.069598,-0.061794,-0.054055,-0.049068,-0.051406,-0.061122,...,-1.088868,-1.083674,-1.077111,-1.070470,-1.064798,-1.061968,-1.061257,-1.061425,-1.061774,-1.062007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0.163143,0.191751,0.162482,0.082889,-0.003124,-0.060718,-0.078752,-0.070726,-0.035320,0.018403,...,-0.130303,-0.195450,-0.215089,-0.198566,-0.160422,-0.117462,-0.072142,-0.020497,0.039363,0.105644
200,-0.659865,-0.598293,-0.519439,-0.433022,-0.353936,-0.296453,-0.265899,-0.260807,-0.282334,-0.315820,...,0.110090,0.056465,0.072822,0.184469,0.363783,0.499735,0.482374,0.310390,0.016343,-0.361188
201,-0.585641,-0.551392,-0.498068,-0.429744,-0.360813,-0.310957,-0.286420,-0.293703,-0.327518,-0.372345,...,-0.099656,-0.136939,-0.051101,0.142426,0.346271,0.418844,0.307861,0.048351,-0.313472,-0.740324
202,0.511181,0.514880,0.517192,0.510124,0.491961,0.468911,0.445530,0.425650,0.412374,0.407223,...,-0.630844,-0.787376,-0.910885,-0.915838,-0.826873,-0.752107,-0.768553,-0.852763,-0.972243,-1.110150


In [21]:
signal_df_for_etiqueta=main_df.filter(items=['Exposicion']).join(signal_df_scaler)
signal_df_for_etiqueta

,Exposicion,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,506,507,508,509,510
0,DD4,-0.163704,-0.226905,-0.298079,-0.367135,-0.421284,-0.457079,-0.484360,-0.516916,-0.548558,...,-1.164791,-1.157399,-1.157358,-1.172348,-1.196807,-1.227327,-1.261004,-1.289491,-1.308634,-1.319887
1,DD4,-0.238344,-0.297405,-0.362885,-0.420639,-0.462113,-0.488852,-0.512144,-0.534643,-0.552667,...,-1.197674,-1.191415,-1.194505,-1.210706,-1.235543,-1.267748,-1.301299,-1.323443,-1.328553,-1.319521
2,DD4,-0.426621,-0.425815,-0.421427,-0.413410,-0.405915,-0.401266,-0.396925,-0.390545,-0.384117,...,-0.809515,-0.809041,-0.806028,-0.801925,-0.799624,-0.800241,-0.799933,-0.797347,-0.792461,-0.785844
3,DD4,-0.031325,-0.024536,-0.016842,-0.008328,-0.000267,0.004768,0.005617,-0.000125,-0.014325,...,-0.978751,-0.978553,-0.974791,-0.967465,-0.957565,-0.945147,-0.930862,-0.918049,-0.908516,-0.901840
4,DD4,-0.071885,-0.076743,-0.078798,-0.076110,-0.069598,-0.061794,-0.054055,-0.049068,-0.051406,...,-1.088868,-1.083674,-1.077111,-1.070470,-1.064798,-1.061968,-1.061257,-1.061425,-1.061774,-1.062007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,LL8,0.163143,0.191751,0.162482,0.082889,-0.003124,-0.060718,-0.078752,-0.070726,-0.035320,...,-0.130303,-0.195450,-0.215089,-0.198566,-0.160422,-0.117462,-0.072142,-0.020497,0.039363,0.105644
200,LL8,-0.659865,-0.598293,-0.519439,-0.433022,-0.353936,-0.296453,-0.265899,-0.260807,-0.282334,...,0.110090,0.056465,0.072822,0.184469,0.363783,0.499735,0.482374,0.310390,0.016343,-0.361188
201,LL8,-0.585641,-0.551392,-0.498068,-0.429744,-0.360813,-0.310957,-0.286420,-0.293703,-0.327518,...,-0.099656,-0.136939,-0.051101,0.142426,0.346271,0.418844,0.307861,0.048351,-0.313472,-0.740324
202,LL8,0.511181,0.514880,0.517192,0.510124,0.491961,0.468911,0.445530,0.425650,0.412374,...,-0.630844,-0.787376,-0.910885,-0.915838,-0.826873,-0.752107,-0.768553,-0.852763,-0.972243,-1.110150


In [22]:
def asign_with_dict(key):
    return asign_labels[key]

In [23]:
signal_df_for_etiqueta['etiqueta']=signal_df_for_etiqueta['Exposicion'].map(asign_with_dict)
signal_df_for_etiqueta.to_excel('signal_df_for_classifier.xlsx')

C:\Users\User\AppData\Local\Temp/ipykernel_11692/295725845.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  signal_df_for_etiqueta['etiqueta']=signal_df_for_etiqueta['Exposicion'].map(asign_with_dict)


In [24]:
stats_df_for_etiqueta['etiqueta']=stats_df_for_etiqueta['Exposicion'].map(asign_with_dict)
stats_df_for_etiqueta

,Exposicion,mean,min,max,std,energy,etiqueta
0,DD4,-0.573769,0.333366,0.015322,-0.089710,-0.289680,0.0
1,DD4,-0.672474,0.242211,0.098394,-0.018877,-0.240977,0.0
2,DD4,-0.652334,-0.418959,1.105769,1.011489,0.720702,0.0
3,DD4,-0.355042,-0.300300,0.382823,0.596810,0.274762,0.0
4,DD4,-1.089536,-2.900485,2.502953,3.157765,4.275857,0.0
...,...,...,...,...,...,...,...
199,LL8,0.253094,1.107195,-0.838128,-0.911431,-0.685505,2.0
200,LL8,-0.143364,0.250275,-1.121410,-0.767125,-0.638139,2.0
201,LL8,-0.119483,0.412712,-1.155641,-0.854059,-0.667660,2.0
202,LL8,0.964187,0.783618,-0.576171,-0.634329,-0.585863,2.0


In [25]:
wave_df_for_etiqueta['etiqueta']=wave_df_for_etiqueta['Exposicion'].map(asign_with_dict)
wave_df_for_etiqueta

,Exposicion,LATENCIA ONDA B,AMPLITUD ONDA B,ANCHO ONDA B,LATENCIA ONDA A,AMPLITUD ONDA A,ANCHO ONDA A,LATENCIA ONDA B FILT.,AMPLITUD ONDA B FILT.,ANCHO ONDA B FILT.,LATENCIA ONDA A FILT.,AMPLITUD ONDA A FILT.,ANCHO ONDA A FILT.,PENDIENTE AB,etiqueta
0,DD4,0.622378,0.306643,0.712617,0.403509,0.313619,0.393907,0.645390,0.245408,0.678310,0.305164,0.341063,0.426210,0.190702,0.0
1,DD4,0.622378,0.333144,0.701994,0.403509,0.315374,0.389673,0.645390,0.255094,0.670636,0.300469,0.356684,0.430768,0.197522,0.0
2,DD4,0.576923,0.571029,0.720466,0.327485,0.002049,0.088827,0.556738,0.393777,0.678594,0.281690,0.261962,0.356294,0.286526,0.0
3,DD4,0.510490,0.351565,0.743069,0.397661,0.275271,0.353079,0.464539,0.254751,0.726874,0.314554,0.401809,0.399988,0.397147,0.0
4,DD4,0.412587,0.743059,0.878371,0.245614,0.506330,0.185957,0.450355,0.631879,0.762267,0.159624,0.528046,0.243415,0.525183,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,LL8,0.534965,0.123091,0.698020,0.555556,0.192956,0.434070,0.776596,0.116198,0.636979,0.469484,0.183700,0.548864,0.088031,2.0
200,LL8,0.311189,0.114917,0.819111,0.111111,0.063570,0.129227,0.730496,0.208037,0.893563,0.070423,0.025261,0.114618,0.048657,2.0
201,LL8,0.307692,0.099042,0.833159,0.105263,0.092043,0.131547,0.734043,0.205093,0.909447,0.075117,0.040668,0.124129,0.050438,2.0
202,LL8,0.318182,0.128317,0.422125,0.064327,0.020192,0.095686,0.379433,0.207628,0.572844,0.070423,0.018486,0.114576,0.111979,2.0


In [26]:
signal_df_short= signal_df_for_etiqueta.copy().dropna() #el dropna lo que hace es quitar donde tenia None en el dict
signal_df_short.to_excel('signal_df_short_for_classifier.xlsx')

In [27]:
stats_df_for_corr = stats_df_for_etiqueta.copy().dropna() #el dropna lo que hace es quitar donde tenia None en el dict
stats_df_for_corr.pop("Exposicion")

0      DD4
1      DD4
2      DD4
3      DD4
4      DD4
      ... 
199    LL8
200    LL8
201    LL8
202    LL8
203    LL8
Name: Exposicion, Length: 169, dtype: object

In [28]:
stats_df_for_corr.index=np.arange(stats_df_for_corr.values.shape[0]) #devuelve matriz de 175 valores, chequear

In [29]:
stats_df_for_corr

,mean,min,max,std,energy,etiqueta
0,-0.573769,0.333366,0.015322,-0.089710,-0.289680,0.0
1,-0.672474,0.242211,0.098394,-0.018877,-0.240977,0.0
2,-0.652334,-0.418959,1.105769,1.011489,0.720702,0.0
3,-0.355042,-0.300300,0.382823,0.596810,0.274762,0.0
4,-1.089536,-2.900485,2.502953,3.157765,4.275857,0.0
...,...,...,...,...,...,...
164,0.253094,1.107195,-0.838128,-0.911431,-0.685505,2.0
165,-0.143364,0.250275,-1.121410,-0.767125,-0.638139,2.0
166,-0.119483,0.412712,-1.155641,-0.854059,-0.667660,2.0
167,0.964187,0.783618,-0.576171,-0.634329,-0.585863,2.0


In [30]:
stats_df_for_corr.to_excel('stats_df_for_corr.xlsx')

In [31]:
wave_df_for_corr = wave_df_for_etiqueta.copy().dropna() #el dropna lo que hace es quitar donde tenia None en el dict
wave_df_for_corr.pop("Exposicion")

0      DD4
1      DD4
2      DD4
3      DD4
4      DD4
      ... 
199    LL8
200    LL8
201    LL8
202    LL8
203    LL8
Name: Exposicion, Length: 169, dtype: object

In [32]:
wave_df_for_corr.index=np.arange(wave_df_for_corr.values.shape[0]) #devuelve matriz de 175 valores, chequear

In [33]:
wave_df_for_corr

,LATENCIA ONDA B,AMPLITUD ONDA B,ANCHO ONDA B,LATENCIA ONDA A,AMPLITUD ONDA A,ANCHO ONDA A,LATENCIA ONDA B FILT.,AMPLITUD ONDA B FILT.,ANCHO ONDA B FILT.,LATENCIA ONDA A FILT.,AMPLITUD ONDA A FILT.,ANCHO ONDA A FILT.,PENDIENTE AB,etiqueta
0,0.622378,0.306643,0.712617,0.403509,0.313619,0.393907,0.645390,0.245408,0.678310,0.305164,0.341063,0.426210,0.190702,0.0
1,0.622378,0.333144,0.701994,0.403509,0.315374,0.389673,0.645390,0.255094,0.670636,0.300469,0.356684,0.430768,0.197522,0.0
2,0.576923,0.571029,0.720466,0.327485,0.002049,0.088827,0.556738,0.393777,0.678594,0.281690,0.261962,0.356294,0.286526,0.0
3,0.510490,0.351565,0.743069,0.397661,0.275271,0.353079,0.464539,0.254751,0.726874,0.314554,0.401809,0.399988,0.397147,0.0
4,0.412587,0.743059,0.878371,0.245614,0.506330,0.185957,0.450355,0.631879,0.762267,0.159624,0.528046,0.243415,0.525183,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,0.534965,0.123091,0.698020,0.555556,0.192956,0.434070,0.776596,0.116198,0.636979,0.469484,0.183700,0.548864,0.088031,2.0
165,0.311189,0.114917,0.819111,0.111111,0.063570,0.129227,0.730496,0.208037,0.893563,0.070423,0.025261,0.114618,0.048657,2.0
166,0.307692,0.099042,0.833159,0.105263,0.092043,0.131547,0.734043,0.205093,0.909447,0.075117,0.040668,0.124129,0.050438,2.0
167,0.318182,0.128317,0.422125,0.064327,0.020192,0.095686,0.379433,0.207628,0.572844,0.070423,0.018486,0.114576,0.111979,2.0


In [34]:
wave_df_for_corr.to_excel('wave_df_for_corr.xlsx')

In [35]:
#correlacion

corr_stats_df=stats_df_for_corr.corr()

corr_stats_df
#la corr de "minimo" con "etiqueta" da positivo porque el minimo tiene en cuenta el "y de picos" y no la amplitud (la cual fue tomada como positiva)

,mean,min,max,std,energy,etiqueta
mean,1.000000,0.425874,-0.142970,-0.304321,-0.305306,0.229586
min,0.425874,1.000000,-0.751563,-0.909475,-0.864513,0.495481
max,-0.142970,-0.751563,1.000000,0.937447,0.880529,-0.695158
std,-0.304321,-0.909475,0.937447,1.000000,0.959835,-0.645496
energy,-0.305306,-0.864513,0.880529,0.959835,1.000000,-0.538529
etiqueta,0.229586,0.495481,-0.695158,-0.645496,-0.538529,1.000000


In [36]:
corr_stats_df.to_excel('corr_stats_df.xlsx')

Lo que hicimos fue asignar etiquetas de acuerdo a los dias, y al control le asignamos 0. Pusimos None a las carpetas de
12_12 led y DP, porque no me dan info taaaaaan importante, y al sacarlos la correlacion mejoro de 50 a 60%.

In [37]:
corr_wave_df=wave_df_for_corr.corr()

corr_wave_df['etiqueta'] # asi vemos la corr que mas nos interesa, es decir, entre etiqueta y los demas features


LATENCIA ONDA B          0.050849
AMPLITUD ONDA B         -0.679188
ANCHO ONDA B            -0.321951
LATENCIA ONDA A          0.080009
AMPLITUD ONDA A         -0.185164
ANCHO ONDA A             0.188071
LATENCIA ONDA B FILT.    0.445851
AMPLITUD ONDA B FILT.   -0.650432
ANCHO ONDA B FILT.      -0.189276
LATENCIA ONDA A FILT.    0.354747
AMPLITUD ONDA A FILT.   -0.336079
ANCHO ONDA A FILT.       0.347467
PENDIENTE AB            -0.628291
etiqueta                 1.000000
Name: etiqueta, dtype: float64

In [38]:
corr_wave_df.to_excel('corr_wave_df.xlsx')
corr_wave_df['etiqueta'].to_excel('corr_wave_df_short.xlsx')

Hacer varias combinaciones de features para armar dataframe. 
En lugar de generar dataframe, otra forma puede ser guardar como numpy.save y lo cargo con numpy.load
y en otro jupyter hacer el aprendizaje
Siempre revisar el escalamiento si lo hace a lo largo de las muestras o de las caracteristicas

Entrar a sklearn y buscar en API todos los que sean clasificadores
En modelos nos va a pedir arreglos np, entonces tenemos que separar el dataframe, tenemos que entregar un np de las etiquetas 
y otro np donde esta todo lo otro que no es etiquetas
opciones: pop, loc o iloc tecnicas para separar el df.

Los metodos se van entrenando fila por fila.
Buscar en google "review on classification methods".